<a href="https://colab.research.google.com/github/ben854719/Threat-Detection-System/blob/main/Agentic_AI_Monitor_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph

  Using cached langchain_google_genai-2.1.10-py3-none-any.whl.metadata (7.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
INFO: This is taking longer than usual. You might need to pr

In [17]:
!pip install "mcp[cli]"

In [18]:
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [19]:
!cd mcp-server-demo
!ls

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory
sample_data


In [20]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory


In [23]:
from ast import Try
from IPython import get_ipython
from IPython.display import display
from mcp.server.fastmcp import FastMCP
import os
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from typing import TypedDict, List
from google.colab import userdata

# Import Google colab.
Colab_Secret_key = "Ben856"

# Import API Key to functions Gemimi.
api_key = userdata.get("Ben856")
if not api_key:
   raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets with the same Ben856")

# Initialize google gemini.
gemini_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', api_key=api_key)

# Define the state of schema using TypeDict.
class LogAnalysisState(TypedDict):
  logs: List[str]
  analysis: str
  translated_text: str

# Analyze a security logs.
def analyze_logs(state: LogAnalysisState) -> dict:
    """
    Analyzes a list of security logs to detect anomalies using the Gemini model.

    Args:
        state: The current state of the LangGraph workflow, containing the logs.

    Returns:
        A dictionary containing the analysis to update the state. The key 'analysis'
        will hold a string with the detailed analysis, potentially including identified
        anomalies and their descriptions.
    """
    logs = state['logs']

    # Construct a more detailed prompt for the model.
    prompt_text = (
        "Analyze the following security logs carefully. Your task is to identify."
        "any anomalies, suspicious patterns, or potential external security threats."
        "For each identified anomaly, provide a brief description and indicate."
        "Detect any security breaches in the user computer."
        "Are the attackers are asking for money from the user before the attackers are leaking confidential information to the public."
        "which log entries are related.\n\n"
        "Security Logs:\n" + "\n".join(logs) + "\n\n"
        "Please provide your analysis in a clear and concise manner."
    )

    # Invoke the Gemini Model with the prompt wrapped in a human message.
    try:
        response = gemini_model.invoke([HumanMessage(content=prompt_text)])
        analysis_result = response.content
    except Exception as e:
        analysis_result = f"Error during analysis:  {e}"
        display(analysis_result)

    # Return a dictionary with the analysis content to update the state.
    return {"analysis": analysis_result}

# Translation medical log of the patient file.
def translate_text(state: LogAnalysisState) -> dict:
   """
   Translate the text to the target language using Gemini.
  _Parameters:
  text(str): The text to be translated from English to French
  target_language(str): The target Language to translate from English To French
  _Returns:
  dict: the translated text
  """
   text_to_translate = state['analysis']
   target_language = "French"

   prompt = f"Translate the following text From English to {target_language}: {text_to_translate}"
   response = gemini_model.invoke([HumanMessage(content=prompt)])
   return {"translated_text": response.content}

# Create LangGraph workflow.
workflow = StateGraph(state_schema=LogAnalysisState)
workflow.add_node("log_analysis", analyze_logs)
workflow.add_node("translation", translate_text)
workflow.add_edge("log_analysis", "translation")
workflow.set_entry_point("log_analysis")

# Compile the workflow.
app = workflow.compile()

# Create a security log.
security_log = [
    "User logging succesful",
    "The user name is Eric Smith",
    "Eric is a manager of a computer stores that specializes in selling softwares and hardwares to medium to large organizations in this city",
    "The city of Eric is Montreal, Quebec, Canada",
    "Any security breaches in the client computer",
    "Are the attackers are demanding money from the user before leaking confidential information from the business to the public?",
    "File accessed by unauthorized user",
    "Unanhorized download of file from the user work email address",
    "The work email address of the client Eric1123@computer.com",
    "Any Anomalies detects in the work phone of the user",
    "Screen-locking ransomware blocks access to your device entirely, often with a full-screen message",
    "The work phone of the user is a Samsung Galaxy S53",
    "System Reboot",
    "Failed logging attempt from unknown IP address 200.123.1.102",
    "Regular system health check passed",
    "Detect if the VPN and the Firewll were not compromised from the attackers",
    "Endpoint Detection and response will trigger an alert to the cybersecurity team on behaalf of their client Eric of the security breaches from the attackers",
]

# Run the workflow.
display("Running log analysis workflow...")
result = app.invoke({"logs": security_log})
display("\nAnalysis Result:")
display(result['analysis'])
display(f"Translated text (French): {result['translated_text']}")

# Print original and translated text
print(f"Original text: {result['analysis']}")
print(f"Translated text (French): {result['translated_text']}")

'Running log analysis workflow...'

'\nAnalysis Result:'

'Based on the provided security logs and statements, here is an analysis of anomalies, suspicious patterns, potential external security threats, and answers to your specific questions:\n\n---\n\n### **Summary of Security Breaches and Threats:**\n\nThe logs indicate a severe and multifaceted security incident targeting Eric Smith, a manager of a computer store. Evidence points to unauthorized access, data exfiltration, and a ransomware attack, likely involving a "double extortion" scheme.\n\n---\n\n### **Identified Anomalies, Suspicious Patterns, and Potential External Security Threats:**\n\n1.  **Unauthorized Data Access and Exfiltration**\n    *   **Description:** An unauthorized user has gained access to Eric\'s files and successfully downloaded data from his work email address. This is a critical data breach, indicating a compromise of Eric\'s account or system, leading to potential loss of confidential business information.\n    *   **Related Log Entries:**\n        *   `File acces

'Translated text (French): Voici la traduction du texte de l\'anglais au français :\n\nBasé sur les journaux de sécurité et les déclarations fournis, voici une analyse des anomalies, des schémas suspects, des menaces de sécurité externes potentielles et des réponses à vos questions spécifiques :\n\n---\n\n### **Résumé des violations et menaces de sécurité :**\n\nLes journaux indiquent un incident de sécurité grave et multifacette ciblant Eric Smith, un gérant de magasin d\'informatique. Les preuves pointent vers un accès non autorisé, une exfiltration de données et une attaque par rançongiciel, impliquant probablement un schéma de "double extorsion".\n\n---\n\n### **Anomalies identifiées, schémas suspects et menaces de sécurité externes potentielles :**\n\n1.  **Accès et exfiltration de données non autorisés**\n    *   **Description :** Un utilisateur non autorisé a accédé aux fichiers d\'Eric et a téléchargé avec succès des données depuis son adresse e-mail professionnelle. Il s\'agit

Original text: Based on the provided security logs and statements, here is an analysis of anomalies, suspicious patterns, potential external security threats, and answers to your specific questions:

---

### **Summary of Security Breaches and Threats:**

The logs indicate a severe and multifaceted security incident targeting Eric Smith, a manager of a computer store. Evidence points to unauthorized access, data exfiltration, and a ransomware attack, likely involving a "double extortion" scheme.

---

### **Identified Anomalies, Suspicious Patterns, and Potential External Security Threats:**

1.  **Unauthorized Data Access and Exfiltration**
    *   **Description:** An unauthorized user has gained access to Eric's files and successfully downloaded data from his work email address. This is a critical data breach, indicating a compromise of Eric's account or system, leading to potential loss of confidential business information.
    *   **Related Log Entries:**
        *   `File accessed